In [ ]:
import os
from fnmatch import fnmatch
import numpy as np
from PIL import Image
from filtering.filters import Median
from feature_extraction.glcm import Glcm
import sklearn
import time
from gmm import Gmm
import pickle
from sklearn import preprocessing

In [ ]:
#Read features of training samples
train_samples_feats_matrix_GLCM = np.load("train_samples_feats_matrix_GLCM.npy")
train_samples_feats_matrix_Gabor = np.load("train_samples_feats_matrix_Gabor.npy")

In [ ]:
#Load features of validation samples
valid_samples_feats_matrix_GLCM = np.load("valid_samples_feats_matrix_GLCM.npy")
valid_samples_feats_matrix_Gabor = np.load("valid_samples_feats_matrix_Gabor.npy")

In [ ]:
#concatenating features 
train_samples_feats_matrix_glcm_gabor = np.concatenate((train_samples_feats_matrix_GLCM,train_samples_feats_matrix_Gabor), axis=1)
valid_samples_feats_matrix_glcm_gabor = np.concatenate((valid_samples_feats_matrix_GLCM,valid_samples_feats_matrix_Gabor), axis=1)

In [ ]:
#Load and concatenate features of test samples
test_samples_feats_matrix_GLCM = np.load("test_samples_feats_matrix_GLCM.npy")
test_samples_feats_matrix_Gabor = np.load("test_samples_feats_matrix_Gabor.npy")
test_samples_feats_matrix_glcm_gabor = np.concatenate((test_samples_feats_matrix_GLCM,test_samples_feats_matrix_Gabor), axis=1)

In [ ]:
gm_model = pickle.load( open( "GMM_['GLCM', 'Gabor']_1000", "rb" ))

In [ ]:
#Best threshold for test data 258.679059501
#Predict labels of the new data points
y_predict = gm_model.predict(test_samples_feats_matrix_glcm_gabor)
#Predict novelty score of new data points
y_predict_score = gm_model.score_samples(test_samples_feats_matrix_glcm_gabor)
tmp = []

x_predict_score = gm_model.score_samples(train_samples_feats_matrix_glcm_gabor)
novelty_thresh = np.amin(x_predict_score)

#If data point belongs to the model then 1 else 0
tmp = np.repeat(1, 10000)
tmp[y_predict_score < novelty_thresh] = 0

In [ ]:
#Ground truth
y1 = np.repeat(1, 6000) #normal
y2 = np.repeat(0, 4000) #abnormal
y = np.concatenate((y1,y2))

In [ ]:
f1_binary = sklearn.metrics.f1_score(y, tmp, pos_label = 0, average = 'binary')
f1_macro = sklearn.metrics.f1_score(y, tmp, average = 'macro')
auc = sklearn.metrics.roc_auc_score(y, y_predict_score)
Math_Cof = sklearn.metrics.matthews_corrcoef(y, tmp) 
tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(y, tmp).ravel()
print ("f1_binary: ", f1_binary, "f1_macro: ", f1_macro, "MAth_cof: ", Math_Cof , " auc: ", auc)

In [ ]:
#Get different thresholds value from ROC with corresponding F1_score and AUC
fpr,tpr,thresh = sklearn.metrics.roc_curve(y, y_predict_score)
for t, thres in enumerate (thresh) :
    tmp = np.repeat(0,10000)
    tmp[y_predict_score >= thres] = 1
    f1 = sklearn.metrics.f1_score(y, tmp, pos_label=0, average = 'binary')
    print ("f1: ", f1, "thres: ", thres)

In [ ]:
import matplotlib.pyplot as pl
import matplotlib
from skimage import io

fpr,tpr,thresh = sklearn.metrics.roc_curve(y,y_predict_score)

f,ax = pl.subplots(1,1)
ax.plot(fpr,tpr,label="GMM")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC curves")
ax.legend(loc="lower right")

io.show()

In [ ]:
f.savefig('GMM_GLCM_Gabor_1000.png')